In [1]:
import numpy as np
import pandas as pd
import librosa
import librosa.display
import matplotlib
import sklearn
import os.path
import sklearn
import os
import pickle

In [2]:
tracks = pd.read_csv("fma_metadata/tracks.csv", index_col=0, header=[0,1])
small = tracks['set','subset'] == 'small'
small_tracks = tracks[small]

In [3]:
small_tracks['track_id'] = small_tracks.index

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [4]:
#small_tracks.head()

album                                                     \
         comments         date_created        date_released engineer   
track_id                                                               
2               0  2008-11-26 01:44:45  2009-01-05 00:00:00      NaN   
5               0  2008-11-26 01:44:45  2009-01-05 00:00:00      NaN   
10              0  2008-11-26 01:45:08  2008-02-06 00:00:00      NaN   
140             1  2008-11-26 01:49:59  2007-05-22 00:00:00      NaN   
141             0  2008-11-26 01:49:57  2009-01-16 00:00:00      NaN   

                                                                           \
         favorites  id                                        information   
track_id                                                                    
2                4   1                                            <p></p>   
5                4   1                                            <p></p>   
10               4   6                                                NaN   
140              1  61  <p>Alec K. Redfearn &amp; The Eyesores: Ellen ...   
141              1  60  <p>A full ensamble of strings, drums, electron...   

                                                       ...    track  \
         listens                        producer tags  ... interest   
track_id                                               ...            
2           6073                             NaN   []  ...     4656   
5           6073                             NaN   []  ...     1933   
10         47632                             NaN   []  ...    54881   
140         1300  Alec K. Refearn, Rob Pemberton   []  ...     1593   
141         1304                             NaN   []  ...      839   

                                                                           \
         language_code                                            license   
track_id                                                                    
2                   en  Attribution-NonCommercial-ShareAlike 3.0 Inter...   
5                   en  Attribution-NonCommercial-ShareAlike 3.0 Inter...   
10                  en  Attribution-NonCommercial-NoDerivatives (aka M...   
140                 en  Attribution-Noncommercial-No Derivative Works ...   
141                 en  Attribution-Noncommercial-No Derivative Works ...   

                                                                    track_id  
         listens lyricist number publisher tags               title           
track_id                                                                      
2           1293      NaN      3       NaN   []                Food        2  
5           1151      NaN      6       NaN   []          This World        5  
10         50135      NaN      1       NaN   []             Freeway       10  
140         1299      NaN      2       NaN   []  Queen Of The Wires      140  
141          725      NaN      4       NaN   []                Ohio      141  

[5 rows x 53 columns]

In [4]:
train = small_tracks[small_tracks['set','split'] == 'training']
val = small_tracks[small_tracks['set','split'] == 'validation']
test = small_tracks[small_tracks['set','split'] == 'test']

In [6]:
test['track','genre_top']

track_id
182                Rock
704       International
705       International
706       International
707       International
708       International
709       International
890                Folk
892                Folk
1066       Experimental
1069       Experimental
1073       Experimental
1075       Experimental
1249       Experimental
1259       Experimental
3624               Rock
4108               Rock
7373       Experimental
7374       Experimental
7375       Experimental
7376       Experimental
7377       Experimental
7378       Experimental
7379       Experimental
7381       Experimental
7383       Experimental
7385       Experimental
7386       Experimental
7388       Experimental
7391       Experimental
              ...      
147024          Hip-Hop
147261          Hip-Hop
148002     Instrumental
148028          Hip-Hop
148031          Hip-Hop
148032          Hip-Hop
148082     Experimental
148430    International
148606              Pop
148607              Pop
148608 

In [6]:
ROOT = "fma_small"

In [7]:
#for root, subdirs, files in os.walk(ROOT):
#    print(root)

fma_small
fma_small/135
fma_small/132
fma_small/104
fma_small/103
fma_small/150
fma_small/102
fma_small/105
fma_small/133
fma_small/134
fma_small/151
fma_small/024
fma_small/023
fma_small/015
fma_small/012
fma_small/079
fma_small/046
fma_small/041
fma_small/048
fma_small/077
fma_small/083
fma_small/084
fma_small/070
fma_small/013
fma_small/014
fma_small/022
fma_small/025
fma_small/071
fma_small/085
fma_small/049
fma_small/082
fma_small/076
fma_small/040
fma_small/078
fma_small/047
fma_small/065
fma_small/091
fma_small/096
fma_small/062
fma_small/054
fma_small/053
fma_small/098
fma_small/038
fma_small/007
fma_small/000
fma_small/009
fma_small/036
fma_small/031
fma_small/052
fma_small/099
fma_small/055
fma_small/063
fma_small/097
fma_small/090
fma_small/064
fma_small/030
fma_small/008
fma_small/037
fma_small/001
fma_small/039
fma_small/006
fma_small/145
fma_small/142
fma_small/129
fma_small/116
fma_small/111
fma_small/118
fma_small/127
fma_small/120
fma_small/143
fma_small/144
fma_small/

In [8]:
# Make spectrogram based on track's id
def get_file(trackid):
    trackid = '{0:0=6d}'.format(trackid) # mp3 file 3 digits long
    folder = trackid[:3] # folder is first 3 digits
    filename = ROOT + '/' + folder + '/' + trackid + '.mp3'
    return filename

def make_spect(trackid):
    filename = get_file(trackid)
    #print(filename)
    y, sr = librosa.load(filename) # (y, sampling rate of y)
    spect = librosa.feature.melspectrogram(y=y, sr=sr,n_fft=2048, hop_length=512)
    spect = librosa.power_to_db(spect, ref=np.max)
    spect = spect.T
    return spect
    
def display_spect(spect):
    librosa.display.specshow(spect)

In [9]:
spect = make_spect(5)

In [12]:
def make_df(df):
    X = np.empty((0, 640, 128))
    y = []
    curr = 0
    for index, row in df.iterrows():
        curr += 1
        if curr == 1000:
            break
        try:
            if curr % 100 == 0:
                print('number', curr)
            trackid = int(row['track_id'])
            curr_genre = str(row[('track', 'genre_top')])
            y.append(curr_genre)
            spect = make_spect(trackid)
            spect = spect[:640, :]
            X = np.append(X, [spect], axis=0)
        except:
            print('unable to process', curr)
            continue
    y = np.array(y)
    print(y)
    return X, y

In [ ]:
print("Processing test set.")
X_test, y_test = make_df(test)

number 200
number 400


In [ ]:
X_test_pickle = open("X_test.pickle","wb")
pickle.dump(X_test, X_test_pickle)
X_test_pickle.close()
y_test_pickle = open("y_test.pickle","wb")
pickle.dump(y_test, y_test_pickle)
y_test_pickle.close()

In [ ]:
print("Processing validation set.")
X_val, y_val = make_df(val)

In [ ]:
X_val_pickle = open("X_val.pickle","wb")
pickle.dump(X_val, X_val_pickle)
X_val_pickle.close()
y_val_pickle = open("y_val.pickle","wb")
pickle.dump(y_val, y_val_pickle)
y_val_pickle.close()

In [ ]:
print("Processing training set.")
X_train, y_train = make_df(train)

In [ ]:
X_train_pickle = open("X_train.pickle","wb")
pickle.dump(X_train, X_train_pickle)
X_train_pickle.close()
y_train_pickle = open("y_train.pickle","wb")
pickle.dump(y_train, y_train_pickle)
y_train_pickle.close()